In [5]:
%pwd

'c:\\Users\\Inshal Munaf\\VsCodeProjects\\Kidney_disease_Classification_MLOPS\\research'

In [6]:
import os 
os.chdir('../')

In [7]:
%pwd

'c:\\Users\\Inshal Munaf\\VsCodeProjects\\Kidney_disease_Classification_MLOPS'

In [8]:
import dagshub
from dotenv import load_dotenv

load_dotenv()

REPO_OWNER = os.getenv('DAGSHUB_REPO_OWNER')
REPO_NAME = os.getenv('DAGSHUB_REPO_NAME')

dagshub.init(repo_owner= REPO_OWNER, repo_name= REPO_NAME, mlflow=True)

❗❗❗ AUTHORIZATION REQUIRED ❗❗❗

c:\miniconda\envs\kidney\lib\site-packages\rich\live.py:256: UserWarning: install "ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')



Open the following link in your browser to authorize the client:
https://dagshub.com/login/oauth/authorize?state=7c59eb72-c459-45eb-a984-f73e2cdcaf78&client_id=32b60ba385aa7cecf24046d8195a71c07dd345d9657977863b52e7748e0f0f28&middleman_request_id=9257aaf0d6df37b031f40bf49edd64067af3b70569626b4801d9356626d8b2ed




Accessing as inshalmunaf

Initialized MLflow to track repo "inshalmunaf/Kidney_disease_Classification_MLOPS"

Repository inshalmunaf/Kidney_disease_Classification_MLOPS initialized!

In [11]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size : list
    params_batch_size : int

In [12]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories, save_json

In [ ]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
    ):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])
    
    
    def get_model_evaluation_config(self) -> EvaluationConfig:
 
        self.evaluation_config = EvaluationConfig(
            path_of_model='artifacts\training\model.h5',
            training_data='artifacts\data_ingestion\KidneyDataset',
            mlflow_uri= {f'https://dagshub.com/{REPO_OWNER}/{REPO_NAME}.mlflow'},
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )

        return EvaluationConfig

In [17]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse

In [ ]:
class Evaluation:
    def __init__(self , config : EvaluationConfig):
        pass

    
    def train_valid_generator(self):
        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split = 0.20
        )

        dataflow_kwargs = dict(
            target_size= self.config.params_image_size[:-1],
            batch_size = self.config.params_batch_size,
            interpolation ='bilinear'
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset='validation',
            shuffle= False,
            **dataflow_kwargs
        )

        @staticmethod
        def load_model(path : Path) -> tf.keras.Model:
            return tf.keras.models.load_model(path)
        
        def evaluation(self):
            self.model = self.load_model(self.config.path_of_model),
            self.valid_generator(),
            self.score = self.model.evaluate(self.valid_generator)
            self.save_score()

        def save_score():
            scores = {'loss' : self.score[0] , 'accuracy' : self.score[1]}
            save_json(path = Path('scores.json') , data = scores)

        def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )
            # Model registry does not work with file store
            if tracking_url_type_store != "file":